In [21]:
import pandas as pd
from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder

In [22]:
def application_train(file_path, null_thereshould, unique_threshould):

    # reading the csv file
    df = pd.read_csv(file_path)

    # to preserve target without droping while preparing data because target is highly imbalanced and have 0s above 90%
    target = df['TARGET']

    # drop TARGET from the dataset
    df.drop('TARGET', inplace=True, axis=1)

    # # generate the profile report
    # prof = ProfileReport(df, title='Data Analysis of application_train.csv', minimal=True)

    # # save the report into a html file
    # prof.to_file('application_train_report.html')

    # find columns containing null values above a threshould and remove them
    df.drop(df.columns[df.isna().sum() / df.shape[0] > null_thereshould], axis=1, inplace=True)

    # find columns containing the same value/category above a threshould and remove them
    df.drop(df.columns[[df[col].value_counts().max() / df.shape[0] > unique_threshould for col in df.columns]], axis=1, inplace=True)

    # adjust incorrect data
    df['DAYS_EMPLOYED'] = abs(df['DAYS_EMPLOYED']) # this cannot contain negatives so replace all with the positives
    df['DAYS_BIRTH'] = abs(df['DAYS_BIRTH'])
    df['DAYS_REGISTRATION'] = abs(df['DAYS_REGISTRATION'])
    df['DAYS_ID_PUBLISH'] = abs(df['DAYS_ID_PUBLISH'])
    df['ORGANIZATION_TYPE'].replace({'Industry: type 1':'Industry',
                                     'Industry: type 2':'Industry',
                                     'Industry: type 3':'Industry',
                                     'Industry: type 4':'Industry',
                                     'Industry: type 5':'Industry',
                                     'Industry: type 6':'Industry',
                                     'Industry: type 7':'Industry',
                                     'Industry: type 8':'Industry',
                                     'Industry: type 9':'Industry',
                                     'Industry: type 10':'Industry',
                                     'Industry: type 11':'Industry',
                                     'Industry: type 12':'Industry',
                                     'Industry: type 13':'Industry',
                                     'Business Entity Type 1':'Business',
                                     'Business Entity Type 2':'Business',
                                     'Business Entity Type 3':'Business',
                                     'Transport: type 1':'Transport',
                                     'Transport: type 2':'Transport',
                                     'Transport: type 3':'Transport',
                                     'Transport: type 4':'Transport',
                                     'Transport: type 5':'Transport',
                                     'Transport: type 6':'Transport',
                                     'Transport: type 7':'Transport',}, inplace=True)

    df_apptrain = pd.concat([df, target], axis=1)

    return df_apptrain

In [23]:
def bureau_balance():

    df = pd.read_csv('data/bureau_balance.csv')

    df = pd.get_dummies(df, dummy_na = True)

    agg_list = {'MONTHS_BALANCE': ['count', 'mean', 'median', 'min', 'max', 'std'],
                    "STATUS_0":["sum"],
                    "STATUS_1":["sum"],
                    "STATUS_2":["sum"],
                    "STATUS_3":["sum"],
                    "STATUS_4":["sum"],
                    "STATUS_5":["sum"],
                    "STATUS_C":["sum"],
                    "STATUS_X":["sum"] }

    bb_agg = df.groupby("SK_ID_BUREAU").agg(agg_list).reset_index()

    # Degisken isimlerinin yeniden adlandirilmasi 
    bb_agg.columns = pd.Index([col[0] + ("_" + col[1].upper() if col[1] else '') for col in bb_agg.columns.tolist()])

    return bb_agg


In [24]:
bureau_balance()

,SK_ID_BUREAU,MONTHS_BALANCE_COUNT,MONTHS_BALANCE_MEAN,MONTHS_BALANCE_MEDIAN,MONTHS_BALANCE_MIN,MONTHS_BALANCE_MAX,MONTHS_BALANCE_STD,STATUS_0_SUM,STATUS_1_SUM,STATUS_2_SUM,STATUS_3_SUM,STATUS_4_SUM,STATUS_5_SUM,STATUS_C_SUM,STATUS_X_SUM
0,5001709,97,-48.0,-48.0,-96,0,28.145456,0,0,0,0,0,0,86,11
1,5001710,83,-41.0,-41.0,-82,0,24.103942,5,0,0,0,0,0,48,30
2,5001711,4,-1.5,-1.5,-3,0,1.290994,3,0,0,0,0,0,0,1
3,5001712,19,-9.0,-9.0,-18,0,5.627314,10,0,0,0,0,0,9,0
4,5001713,22,-10.5,-10.5,-21,0,6.493587,0,0,0,0,0,0,0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
817390,6842884,48,-23.5,-23.5,-47,0,14.000000,9,0,0,0,0,0,20,19
817391,6842885,24,-11.5,-11.5,-23,0,7.071068,12,0,0,0,0,12,0,0
817392,6842886,33,-16.0,-16.0,-32,0,9.669540,8,0,0,0,0,0,25,0
817393,6842887,37,-18.0,-18.0,-36,0,10.824355,6,0,0,0,0,0,31,0


In [25]:
def bureau_bb(unique_threshould, null_thereshould):
    
    df_bb = bureau_balance()

    # read bureau.csv
    df_bu = pd.read_csv('data/bureau.csv')
    
    df = df_bu.merge(df_bb, how='left', on='SK_ID_BUREAU')
    
    df['CREDIT_TYPE'] = df['CREDIT_TYPE'].replace([
                  'Car loan',
                  'Mortgage',
                  'Microloan',
                  'Loan for business development', 
                  'Another type of loan',
                  'Unknown type of loan', 
                  'Loan for working capital replenishment',
                  "Loan for purchase of shares (margin lending)",                                                
                  'Cash loan (non-earmarked)', 
                  'Real estate loan',
                  "Loan for the purchase of equipment", 
                  "Interbank credit", 
                  "Mobile operator loan"], 'Rare')
    
    df['CREDIT_ACTIVE'] = df['CREDIT_ACTIVE'].replace(['Bad debt','Sold'], 'Active')
        
    df = pd.get_dummies(df, columns = ["CREDIT_TYPE","CREDIT_ACTIVE"])
        
    df.drop(["CREDIT_CURRENCY", "SK_ID_BUREAU"], inplace = True, axis = 1)
    
    df["MONTHS_CREDIT"] = round((df.DAYS_CREDIT_ENDDATE - df.DAYS_CREDIT)/30)
    
    agg_list = {
              'MONTHS_BALANCE_COUNT': ['count'],
              'MONTHS_BALANCE_MEAN': ['mean'],
              'MONTHS_BALANCE_MEDIAN': ['median'],
              'MONTHS_BALANCE_MIN':['min'],
              'MONTHS_BALANCE_MAX':['max'], 
              'MONTHS_BALANCE_STD':['std'],
              "SK_ID_CURR":["count"],
              "DAYS_CREDIT":["min","max"],
              "CREDIT_DAY_OVERDUE":["sum","mean","max"],     
              "DAYS_CREDIT_ENDDATE":["max","min"],
              "DAYS_ENDDATE_FACT":["max","min"],
              "AMT_CREDIT_MAX_OVERDUE":["mean","max","min"],
              "CNT_CREDIT_PROLONG":["sum","mean","max","min"],
              "AMT_CREDIT_SUM":["mean","max","min"],            
              "AMT_CREDIT_SUM_DEBT":["sum","mean","max"],
              "AMT_CREDIT_SUM_LIMIT":["sum","mean","max"],
              'AMT_CREDIT_SUM_OVERDUE':["sum","mean","max"], 
              'DAYS_CREDIT_UPDATE':["max","min"],
              'AMT_ANNUITY':["sum","mean"],
              'MONTHS_BALANCE_COUNT':["sum"], 
              'STATUS_0_SUM':["mean"],
              'STATUS_1_SUM':["mean"],
              'STATUS_2_SUM':["mean"],
              'STATUS_3_SUM':["mean"],
              'STATUS_4_SUM':["mean"],
              'STATUS_5_SUM':["mean"],
              'STATUS_X_SUM':["mean"],
              'STATUS_C_SUM':["mean"], 
              'CREDIT_ACTIVE_Active':["sum","mean"], 
              'CREDIT_ACTIVE_Closed':["sum","mean"], 
              'CREDIT_TYPE_Rare':["sum","mean"],      
              'CREDIT_TYPE_Consumer credit':["sum","mean"], 
              'CREDIT_TYPE_Credit card':["sum","mean"],
              "MONTHS_CREDIT":["count","sum","mean","max","min"]
        }
    
    bureau_agg = df.groupby("SK_ID_CURR").agg(agg_list).reset_index()
    bureau_agg.columns = pd.Index([col[0] + ("_" + col[1].upper() if col[1] else '') for col in bureau_agg.columns.tolist()])

    # find columns containing null values above a threshould and remove them
    bureau_agg.drop(bureau_agg.columns[bureau_agg.isna().sum() / bureau_agg.shape[0] > null_thereshould], axis=1, inplace=True)

    # find columns containing the same value/category above a threshould and remove them
    bureau_agg.drop(bureau_agg.columns[[bureau_agg[col].value_counts().max() / bureau_agg.shape[0] > unique_threshould for col in bureau_agg.columns]], axis=1, inplace=True)
    
    return bureau_agg

    

In [26]:
def combine():

    at = application_train(file_path='data/application_train.csv', null_thereshould=0.5, unique_threshould=0.9)
    bu_bb = bureau_bb(unique_threshould=0.9, null_thereshould=0.5)

    at_bu_bb = at.merge(bu_bb, how='left', on='SK_ID_CURR')

    return at_bu_bb

In [27]:
c = combine()
c

C:\Users\Eshan\AppData\Local\Temp\ipykernel_2876\3348335035.py:29: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['ORGANIZATION_TYPE'].replace({'Industry: type 1':'Industry',


,SK_ID_CURR,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,...,CREDIT_TYPE_Rare_MEAN,CREDIT_TYPE_Consumer credit_SUM,CREDIT_TYPE_Consumer credit_MEAN,CREDIT_TYPE_Credit card_SUM,CREDIT_TYPE_Credit card_MEAN,MONTHS_CREDIT_COUNT,MONTHS_CREDIT_SUM,MONTHS_CREDIT_MEAN,MONTHS_CREDIT_MAX,MONTHS_CREDIT_MIN
0,100002,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,...,0.0,4.0,0.500000,4.0,0.500000,6.0,144.0,24.00,61.0,3.0
1,100003,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,...,0.0,2.0,0.500000,2.0,0.500000,4.0,115.0,28.75,61.0,5.0
2,100004,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,...,0.0,2.0,1.000000,0.0,0.000000,2.0,25.0,12.50,24.0,1.0
3,100006,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,...,0.0,1.0,1.000000,0.0,0.000000,1.0,12.0,12.00,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,M,N,N,0,157500.0,254700.0,27558.0,225000.0,Unaccompanied,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,F,N,Y,0,72000.0,269550.0,12001.5,225000.0,Unaccompanied,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,F,N,Y,0,153000.0,677664.0,29979.0,585000.0,Unaccompanied,...,0.0,3.0,0.750000,1.0,0.250000,4.0,153.0,38.25,61.0,24.0
307509,456254,F,N,Y,0,171000.0,370107.0,20205.0,319500.0,Unaccompanied,...,0.0,1.0,1.000000,0.0,0.000000,1.0,8.0,8.00,8.0,8.0


In [28]:
c.to_csv('data/all_data.csv')

In [29]:
d = c.select_dtypes(include=['float64', 'int64'])

In [44]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
import matplotlib.pyplot as plt

def mahalanobis_distance(df):
    mean = np.mean(df, axis=0)
    cov_matrix = np.cov(df, rowvar=False)
    inv_cov_matrix = np.linalg.inv(cov_matrix)
    distances = []
    for i in range(df.shape[0]):
        row = df.iloc[i, :] - mean
        distance = np.sqrt(np.dot(np.dot(row, inv_cov_matrix), row.T))
        distances.append(distance)
    return np.array(distances)

def identify_outliers_mahalanobis(df, threshold=0.01):
    distances = mahalanobis_distance(df)
    chi2_threshold = chi2.ppf((1 - threshold), df.shape[1])
    dists = distances.copy()
    return distances > chi2_threshold, distances

outliers = identify_outliers_mahalanobis(d)
df_cleaned = d[~outliers[0]]


In [40]:
df_cleaned

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,CREDIT_TYPE_Rare_MEAN,CREDIT_TYPE_Consumer credit_SUM,CREDIT_TYPE_Consumer credit_MEAN,CREDIT_TYPE_Credit card_SUM,CREDIT_TYPE_Credit card_MEAN,MONTHS_CREDIT_COUNT,MONTHS_CREDIT_SUM,MONTHS_CREDIT_MEAN,MONTHS_CREDIT_MAX,MONTHS_CREDIT_MIN
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,9461,637,3648.0,...,0.0,4.0,0.500000,4.0,0.500000,6.0,144.0,24.00,61.0,3.0
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,16765,1188,1186.0,...,0.0,2.0,0.500000,2.0,0.500000,4.0,115.0,28.75,61.0,5.0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,19046,225,4260.0,...,0.0,2.0,1.000000,0.0,0.000000,2.0,25.0,12.50,24.0,1.0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,19005,3039,9833.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,19932,3038,4311.0,...,0.0,1.0,1.000000,0.0,0.000000,1.0,12.0,12.00,12.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,157500.0,254700.0,27558.0,225000.0,0.032561,9327,236,8456.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252,0,72000.0,269550.0,12001.5,225000.0,0.025164,20775,365243,4388.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253,0,153000.0,677664.0,29979.0,585000.0,0.005002,14966,7921,6737.0,...,0.0,3.0,0.750000,1.0,0.250000,4.0,153.0,38.25,61.0,24.0
307509,456254,0,171000.0,370107.0,20205.0,319500.0,0.005313,11961,4786,2562.0,...,0.0,1.0,1.000000,0.0,0.000000,1.0,8.0,8.00,8.0,8.0
